In [2]:
from modules.utils import *
from modules.YfScrapper import *
pd.set_option('display.max_columns', None)

In [ ]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO
from collections.abc import Iterable 
from modules.utils import logger

class YfScrapper():
    '''
    Scrapper object to get data from Yahoo Finance, can contain multiple data for different tickers
    '''    
    def __init__(self):
        '''
        Sets the headers to be used
        '''
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Accept-Language': 'en-US,en;q=0.5',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
            }
        self.mapping_dict = {
            'Market Cap (intraday)' : 'Market Cap (B)',
            'Enterprise Value' : 'Enterprise Value (B)',
            '52-Week Change' : '52 Week Change (%)',
            'S&P500 52-Week Change': 'S&P500 52-Week Change (%)',
            'Avg Vol 3 month' : 'Avg Vol 3 month (B)',
            'Avg Vol (10 day)' : 'Avg Vol 10 day (B)',
            'Shares Short' : 'Shares Short (M) (prior month)',
            'Forward Annual Dividend Yield': 'Forward Annual Dividend Yield (%)',
            'Trailing Annual Dividend Yield' : 'Trailing Annual Dividend Yield (%)',
            'Payout Ratio' : 'Payout Ratio (%)',
            'Last Split Factor' : 'Last Split Factor (x:1)',
            'Profit Margin' : 'Profit Margin (%)',
            'Operating Margin (ttm)' : 'Operating Margin (ttm) (%)',
            'Return on Assets (ttm)' : 'Return on Assets (ttm) (%)',
            'Return on Equity (ttm)' : 'Return on Equity (ttm) (%)',
            'Revenue (ttm)' : 'Revenue (ttm) (B)',
            'Quarterly Revenue Growth (yoy)' : 'Quarterly Revenue Growth (yoy) (%)',
            'Gross Profit (ttm)' : 'Gross Profit (ttm) (B)',
            'EBITDA' : 'EBITDA (B)',
            'Net Income Avi to Common (ttm)' : 'Net Income Avi to Common (ttm) (B)',
            'Quarterly Earnings Growth (yoy)' : 'Quarterly Earnings Growth (yoy) (%)',
            'Total Cash (mrq)' :'Total Cash (mrq) (B)',
            'Total Debt (mrq)' :'Total Debt (mrq) (B)',
            'Operating Cash Flow (ttm)' : 'Operating Cash Flow (ttm) (B)',
            'Levered Free Cash Flow (ttm)' : 'Levered Free Cash Flow (ttm) (B)'
        }
        self.tickers = {}
        self.compiled_dataframes = None

    def add_tickers(self, tickers):
        '''
        Adds a list of tickers to the class instance
        Parameters:
            tickers (list[str] or str) : the ticker symbols to be added

        '''
        if isinstance(tickers, str):
            self.tickers.setdefault(ticker)
        elif isinstance(tickers, Iterable):
            count = 0
            for ticker in tickers:
                self.tickers.setdefault(ticker)
                count +=1
            logger.info(f'Added {count} tickers')
        else:
            raise TypeError('tickers must be str or iterable list of strings')
    
    def _mapper(self, row):
            '''Helper function to map or return original values'''
            return self.mapping_dict.get(row, row)

    def get_ticker_stats(self, tickers, clean_df=True):  
        '''
        Function take takes in a list of tickers and scraps the yahoo stats into a dictionary.
        Parameters:
            tickers (str or iterable list of strings): 
                If 'all', will scrap for all the stored tickers, otherwise provide a list of tickers to scrap or a ticker
            clean_df (bool):
                option whether to clean the data
        '''
        if isinstance(tickers, str):
            if tickers.upper() == 'ALL':
                tickers = self.tickers
            elif tickers.upper().startswith('S&P'):
                tickers = self.get_SP500_data(tickers_only=True)
            else:
                tickers = list(tickers)
        elif isinstance(tickers, Iterable):
            pass
        else:
            raise TypeError('tickers must be str or iterable list of strings')
        
        for ticker in tickers:
            url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
            resp = requests.get(url, headers = self.headers)
            logger.info(f'{ticker} status - {resp.status_code}')
            soup = BeautifulSoup(resp.text, "html.parser")
            name = soup.find("h1").text

            # Read the html using pandas to parse tables directly, then concatenate them
            dfs = pd.read_html(StringIO(resp.text))
            df = pd.concat([*dfs])
            df.columns = ['metrics', ticker]
            # Header cleaning
            df['metrics'].replace(regex={r'[0-9]$': ''}, inplace = True) # Removes the annotations appearing at the end of rows
            df['metrics'].replace(regex={r'(\(.+,.+\))': ''}, inplace = True) # This will specifically remove dates inside brackets, by checking for ','
            ### UNRESOLVED, there are tow columns shares short, the latter is for prior month
            df['metrics'] = df['metrics'].str.strip()
            df['metrics'] = df['metrics'].apply(self._mapper)
            df = df.T
            df.columns = df.iloc[0,:] # Update the first row as the header
            df.insert(0, 'Name', name)
            df = df.drop('metrics') # Drop the first row
            idx = df.columns.to_list().index('Shares Short (M) (prior month)')
            updated_columns = df.columns.to_list()
            updated_columns[idx] = 'Shares Short (M)'
            df.columns = updated_columns
            if clean_df:
                df = self.clean_df(df)
            logger.info(f'\t{df.iloc[0,0]} : {df.iloc[0,1]}')
            # Save to the object variable
            self.tickers[ticker] = df

    def clean_df(self, df):
        '''
        Function to cast and clean the dataframe via the following:
        1. Format strings into numbers according (large number format)
        2. Clean stocksplit ratios 
        3. Recast dates into datetime format

        Parameters:
            df (pd.DataFrame) - dataframe for cleaning
        '''
        for col in df.columns:
            if col in ['Dividend Date', 'Ex-Dividend Date', 'Last Split Date', 'Fiscal Year Ends', 'Most Recent Quarter (mrq)']:
                df[col] = df[col].apply(self._date_conversion)
            elif col == 'Last Split Factor (x:1)':
                df[col] = df[col].apply(self._stocksplits)
            else:
                df[col] = df[col].apply(self._num_reformat)
        return df

    def _num_reformat(self, x):
        '''
        Helper function to reformat large sums to be in Billions and removing % and commas
        Casts numerical values into float type
        '''
        if isinstance(x, str):
            x = re.sub("[,]", "", x)
            if x[-1] == 'T':
                x = round(float(x[:-1])*1000,2)
            elif x[-1] == 'B':
                x = round(float(x[:-1]),2)
            elif x[-1] == 'M':
                x = round(float(x[:-1])*0.001,2)
            elif x[-1] == 'k':
                x = round(float(x[:-1])*0.000001,2)
            elif x[-1] == '%':
                x = round(float(x[:-1]),2)             
            elif x == "N/A":
                x = 0
        return x

    def _stocksplits(self, x):
        '''
        Changes split factors into x:1 whole ratios
        '''
        if isinstance(x, str):
            x = x.split(':')
            return round(int(x[0])/ int(x[1]),2)
        else:
            return x
    
    def _date_conversion(self, x):
        '''
        Parses dates into datetime format at the end of the dataframe
        '''
        if isinstance(x, str):
            # Dec 30, 2022
            return datetime.strptime(x, '%b %d, %Y').date()
        else:
            return x
    
    def compile_dataframes(self):
        '''
        Function to concatenate all the ticker dataframe together
        '''
        dfs = [ticker for ticker in self.tickers.values() if isinstance(ticker, pd.DataFrame)]
        df = pd.concat([*dfs])
        self.compiled_dataframes = df
        return df
        
    def get_SP500_data(self, tickers_only: bool=False, url: str='https://www.slickcharts.com/sp500', tableclass: str ="table-responsive"):
        '''
        Function to scrap the latest S&P data from a website containing S&P data
        Inputs:
            tickers_only: boolean - whether to return the ticker symbols only, or the whole dataframe
            url: string - website url
            tableclass: string - tableclass containing the data
        Returns:
            pd.DataFrame or list
        '''
        resp = requests.get(url, headers = self.headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        table = soup.find(class_ = tableclass)

        table_head = table.find('thead')
        header_list = [th.text.strip() for th in table_head.find_all('th')]

        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        sp_data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            sp_data.append([ele for ele in cols if ele]) # Get rid of empty values

        sp_df = pd.DataFrame(np.array(sp_data))
        sp_df.columns = header_list
        sp_df = sp_df.drop('#', axis=1)
        sp_df['Symbol'].replace(regex={r'[\.]': '-'}, inplace=True) #tickers need to have - instead of . for proper search on yahoo
        logger.info(f'Number of S&P constituent data obtained: {len(sp_df)}')
        if tickers_only:
            return sp_df['Symbol'].to_list()
        else:
            return sp_df

    def to_csv(self, filepath):
        if self.compiled_dataframes:
            self.compiled_dataframes.to_csv(filepath + '.csv')
            logger.info(f'File {filepath} saved!')



<h2>Part 2 - S&P and portfolio web scraper</h2>
<h3>Introduction</h3>
<p>The Standard and Poor's 500, or simply the S&P 500, is a stock market index tracking the stock performance of 500 large companies listed on stock exchanges in the United States. Having consisting of 11 different sectors and over 500 different companies, the index can be used as a benchmark for a basic stock portfolio diversification in the US stock marker. It is one of the most commonly followed equity indices.</p>
    
<p>This script scraps data from the yahoo finance statistics page.</p>
<img src="images/sample.JPG">
<p>Two sets of data are in focus</p>
<ol>
    <li>The S&P index constituents.</li>
    <li>One's portfolio</li>
    </ol>

### Option 1 - S&P data

In [3]:
scrapper = YfScrapper()
sp_data = scrapper.get_SP500_data()
sp_data

2024-01-06 23:03:30,835 - root - INFO - Number of S&P constituent data obtained: 503


,Company,Symbol,Portfolio%,Price,Chg,% Chg
0,Microsoft Corp,MSFT,6.95%,367.75,-0.19,(-0.05%)
1,Apple Inc.,AAPL,6.76%,181.18,-0.73,(-0.40%)
2,Amazon.com Inc,AMZN,3.34%,145.24,0.67,(0.46%)
3,Nvidia Corp,NVDA,3.01%,490.97,10.99,(2.29%)
4,Alphabet Inc. Class A,GOOGL,2.05%,135.73,-0.66,(-0.48%)
...,...,...,...,...,...,...
498,Davita Inc.,DVA,0.01%,107.24,1.06,(1.00%)
499,V.F. Corporation,VFC,0.01%,16.90,-0.04,(-0.24%)
500,"Mohawk Industries, Inc.",MHK,0.01%,103.88,3.17,(3.15%)
501,Fox Corporation Class B,FOX,0.01%,28.10,0.11,(0.39%)


### Option 2 - Personal portfolio data

In [ ]:
input_tickers = pd.read_csv('portfolio_tickers.csv') #input file with a list of portfolio tickers
tickers = input_tickers['Tickers'].tolist()
scrapper.add_tickers(tickers)
scrapper.get_ticker_stats('all', clean_df=True)

In [ ]:
scrapper.get_ticker_stats('S&P500')

In [11]:
final_df = scrapper.compile_data_to_df()
final_df

,Name,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta (5Y Monthly),52 Week Change (%),S&P500 52-Week Change (%),52 Week High,52 Week Low,50-Day Moving Average,200-Day Moving Average,Avg Vol (3 month),Avg Vol 10 day (B),Shares Outstanding,Implied Shares Outstanding,Float,% Held by Insiders,% Held by Institutions,Shares Short (M),Short Ratio,Short % of Float,Short % of Shares Outstanding,Shares Short (M) (prior month),Forward Annual Dividend Rate,Forward Annual Dividend Yield (%),Trailing Annual Dividend Rate,Trailing Annual Dividend Yield (%),5 Year Average Dividend Yield,Payout Ratio (%),Dividend Date,Ex-Dividend Date,Last Split Factor (x:1),Last Split Date,Fiscal Year Ends,Most Recent Quarter (mrq),Profit Margin (%),Operating Margin (ttm) (%),Return on Assets (ttm) (%),Return on Equity (ttm) (%),Revenue (ttm) (B),Revenue Per Share (ttm),Quarterly Revenue Growth (yoy) (%),Gross Profit (ttm) (B),EBITDA (B),Net Income Avi to Common (ttm) (B),Diluted EPS (ttm),Quarterly Earnings Growth (yoy) (%),Total Cash (mrq) (B),Total Cash Per Share (mrq),Total Debt (mrq) (B),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B)
MSFT,Microsoft Corporation (MSFT),2730.00,2670.00,35.63,32.89,2.19,12.58,12.38,12.25,23.88,0.88,61.92,20.69,384.30,226.41,366.76,331.92,0.03,0.02,7.43,7.43,7.43,0.05,73.01,0.05,1.77,0.72,0.72,0.05,3,0.82,2.79,0.76,1.00,26.36,2024-03-13,2024-02-13,2.00,2003-02-17,2023-06-29,2023-09-29,35.31,47.59,14.72,39.11,218.31,29.35,12.8,NaN,109.48,77.10,10.33,27.0,143.95,19.37,105.68,47.88,1.66,29.70,94.97,51.01
AAPL,Apple Inc. (AAPL),2820.00,2870.00,29.56,27.47,2.17,7.47,45.34,7.48,22.20,1.29,39.21,20.69,199.62,128.12,187.40,180.02,0.05,0.05,15.55,15.75,15.54,0.07,61.50,0.12,2.29,0.77,0.77,0.11,0.96,0.53,0.94,0.52,0.80,15.33,2023-11-15,2023-11-09,4.00,2020-08-30,2023-09-29,2023-09-29,25.31,30.13,20.26,171.95,383.29,24.34,-0.7,170.78,125.82,97.00,6.12,10.8,61.55,3.96,123.93,199.42,0.99,4.00,110.54,82.18
AMZN,Amazon.com Inc. (AMZN),1500.00,1570.00,76.04,37.74,2.36,2.72,8.20,2.84,21.38,1.16,66.25,20.69,155.63,87.08,144.64,128.69,0.05,0.04,10.33,10.33,9.06,9.62,61.80,0.09,1.75,1.15,0.87,0.09,NaN,NaN,0.00,0.00,NaN,0.00,NaN,NaN,20.00,2022-06-05,2022-12-30,2023-09-29,3.62,7.82,3.61,12.53,554.03,53.95,12.6,225.15,73.98,20.08,1.92,244.0,64.17,6.21,166.06,90.76,0.99,17.71,71.65,36.39
NVDA,NVIDIA Corporation (NVDA),1210.00,1210.00,64.77,24.81,0.49,27.24,36.46,26.87,52.86,1.64,214.16,20.69,505.48,151.41,471.39,410.45,0.04,0.03,2.47,2.53,2.37,4.04,68.41,0.03,0.6,1.08,1.04,0.03,0.16,0.03,0.16,0.03,0.15,2.11,2023-12-27,2023-12-04,4.00,2021-07-19,2023-01-28,2023-10-28,42.10,57.49,27.23,69.17,44.87,18.18,205.5,15.36,22.16,18.89,7.56,1259.3,18.28,7.4,11.03,33.15,3.59,13.49,18.84,14.11
GOOGL,Alphabet Inc. (GOOGL),1710.00,1620.00,26.00,20.37,1.28,5.85,6.22,5.44,17.32,1.05,54.20,20.69,142.68,85.83,133.61,125.62,0.03,0.02,5.92,12.59,10.79,0.28,79.70,0.05,1.66,0.84,0.40,0.05,NaN,NaN,0.00,0.00,NaN,0.00,NaN,NaN,20.00,2022-07-17,2022-12-30,2023-09-29,22.46,27.96,13.49,25.33,297.13,23.34,11.0,156.63,96.07,66.73,5.23,41.5,119.94,9.58,30.45,11.14,2.04,21.78,106.44,70.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FMC,FMC Corporation (FMC),7.58,11.37,12.93,14.06,NaN,1.54,2.30,2.29,10.36,0.89,NaN,NaN,133.37,49.49,56.36,86.4,0.00,0.00,0.12,NaN,0.12,0.78,92.72,0.01,3.15,5.02,4.43,0.01,2.32,3.82,2.32,3.85,1.89,49.36,2024-01-17,2023-12-27,1.15,2019-03-03,2022-12-30,2023-09-29,10.01,13.14,5.61,18.45,4.96,39.64,-28.7,2.33,1.16,0.59,4.70,NaN,0.32,2.6,4.26,128.63,1.54,26.38,-0.06,-0.21
XRAY,DENTSPLY SIRONA Inc. (XRAY),7.50,9.31,126.19,17.06,2.21,1.92,2.22,2.37,49

# OLD CODE

In [ ]:
# from modules.custom_functions import *

# OLD CODE
# scrapped_data = {}

# try:
#     missed_tickers = scrap_ticker(tickers, data_dict=scrapped_data, sleeptime=2, batch_interval=50)
#     if missed_tickers:
#         missed_tickers = rescrap_missed(missed_tickers, data_dict=scrapped_data, max_tries=2)
# except Exception as e:
#     print(e)
# finally:
#     final_df, ticker_df = clean_df(scrapped_data, metrics, display_progress=False, market_indices=market_indices)
#     final_df.reset_index(drop=True, inplace=True)
# final_df

In [41]:
# filename = input("Enter filename to save as: ")
# if filename == '':
#     filename = 'portfolio'
# current_date = date.today().isoformat()
# final_df.to_csv(f'data/{filename}_{current_date}.csv')
# print(f'Saved to file: {filename}_{current_date}.csv')

Saved to file: s&p_2023-04-16.csv


In [17]:
# import mysql.connector
# import json

# def get_value_from_json(json_file, key, sub_key=None):
#    '''
#    Function to read the json file for our app secret key
#    '''
#    try:
#        with open(json_file) as f:
#            data = json.load(f)
#            if sub_key:
#                return data[key][sub_key]
#            else:
#                return data[key]
#    except Exception as e:
#        print("Error: ", e)

# config = get_value_from_json("data/secrets.json", "mysql_connector")

# # Connect to server on localhost
# try:
#     cnx = mysql.connector.connect(**config)
#     print('Connected to database')
#     cur = cnx.cursor()
#     insert_query = "INSERT IGNORE INTO companies (`ticker`, `name`, `index`) VALUES (%s, %s, %s)"
#     for index, row in ticker_df.iterrows():
#         try:
#             data = (row['ticker'], row['name'].strip(), row['index'])
#             #print(f'Inserting {data}', end = ' ')
#             cur.execute(insert_query, data)
#             cnx.commit()
#             #print('Done')
#         except mysql.connector.Error as err:
#             print(err)
# except mysql.connector.Error as err:
#     print(err)
# finally:
#     print('Completed')
#     cur.close()
#     cnx.close()

Connected to database
1062: Duplicate entry 'TSLA' for key 'PRIMARY'
1062: Duplicate entry 'NVDA' for key 'PRIMARY'
1062: Duplicate entry 'GOOG' for key 'PRIMARY'
1062: Duplicate entry 'AMZN' for key 'PRIMARY'
1062: Duplicate entry 'IDXX' for key 'PRIMARY'
1062: Duplicate entry 'CDNS' for key 'PRIMARY'
1062: Duplicate entry 'MRNA' for key 'PRIMARY'
Completed


In [46]:
# try:
#     cnx = mysql.connector.connect(**config)
#     print('Connected to database')
#     cur = cnx.cursor()
#     insert_query = '''INSERT IGNORE INTO statistics (Ticker, `Market Cap (B)`, `Enterprise Value (B)`, `Trailing P/E`,
#        `Forward P/E`, `PEG Ratio (5 yr expected)`, `Price/Sales (ttm)`,
#        `Price/Book (mrq)`, `Enterprise Value/Revenue`,
#        `Enterprise Value/EBITDA`, `Beta (5Y Monthly)`, `52 Week Change (%)`,
#        `S&P500 52-Week Change (%)`, `52 Week High`, `52 Week Low`,
#        `50-Day Moving Average`, `200-Day Moving Average`,
#        `Avg Vol 3 month (M)`, `Avg Vol 10 day (M)`, `Shares Outstanding`,
#        `Implied Shares Outstanding`, `Float`, `% Held by Insiders`,
#        `% Held by Institutions`, `Shares Short (M)`, `Short Ratio (M)`,
#        `Short % of Float`, `Short % of Shares Outstanding`, `Shares Short`,
#        `Forward Annual Dividend Rate`, `Forward Annual Dividend Yield (%)`,
#        `Trailing Annual Dividend Rate`, `Trailing Annual Dividend Yield (%)`,
#        `5 Year Average Dividend Yield`, `Payout Ratio (%)`,
#        `Last Split Factor (x:1)`, `Profit Margin (%)`,
#        `Operating Margin (ttm) (%)`, `Return on Assets (ttm) (%)`,
#        `Return on Equity (ttm) (%)`, `Revenue (ttm) (B)`,
#        `Revenue Per Share (ttm)`, `Quarterly Revenue Growth (yoy) (%)`,
#        `Gross Profit (ttm) (B)`, `EBITDA (B)`,
#        `Net Income Avi to Common (ttm) (B)`, `Diluted EPS (ttm)`,
#        `Quarterly Earnings Growth (yoy) (%)`, `Total Cash (mrq) (B)`,
#        `Total Cash Per Share (mrq)`, `Total Debt (mrq) (B)`,
#        `Total Debt/Equity (mrq)`, `Current Ratio (mrq)`,
#        `Book Value Per Share (mrq)`, `Operating Cash Flow (ttm) (B)`,
#        `Levered Free Cash Flow (ttm) (B)`, `Dividend Date`, `Ex-Dividend Date`,
#        `Last Split Date`, `Fiscal Year Ends`, `Most Recent Quarter (mrq)`) 
#        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#        %s)'''
#     for index, row in final_df.iterrows():
#         try:
#             data = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], 
#                 row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], 
#                 row[19], row[20], row[21], row[22], row[23], row[24], row[25], row[26], row[27], 
#                 row[28], row[29], row[30], row[31], row[32], row[33], row[34], row[35], row[36], 
#                 row[37], row[38], row[39], row[40], row[41], row[42], row[43], row[44], row[45], 
#                 row[46], row[47], row[48], row[49], row[50], row[51], row[52], row[53], row[54], 
#                 row[55], row[56], row[57], row[58], row[59], row[60])
#             #print(f'Inserting {data}', end = ' ')
#             cur.execute(insert_query, data)
#             cnx.commit()
#             #print('Done')
#         except mysql.connector.Error as err:
#             print(err)

# except mysql.connector.Error as err:
#     print(err)
# finally:
#     print('Completed')
#     cur.close()
#     cnx.close()

Connected to database
1062: Duplicate entry '2023-04-16-AAPL' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-MSFT' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-AMZN' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-NVDA' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-GOOG' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-TSLA' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-CDNS' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-MRNA' for key 'PRIMARY'
1062: Duplicate entry '2023-04-16-IDXX' for key 'PRIMARY'
Completed
